In [1]:
import pymongo
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

In [2]:
client = MongoClient()
db = client.companies
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

### Based on the information provided, we are going to select a possible location for the office based on the available information.

The applied filter assums a **number of emplooyees of 90**. 

The reason? Currently, a huge number of companies have implanted a ***work from home policy*** but, additionally the company ***expect to increase their employees number in the future***, so being continously changing the location would be counterproductive.

Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [3]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city":{"$ne": "",}}
project1 = {"_id":0,"offices.city": 1,"number_of_employees": 1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)#.skip(1).limit(1)
results1

In [4]:
pd.DataFrame.from_dict(results1)

,number_of_employees,offices
0,90,{'city': 'Amsterdam'}
1,90,{'city': 'San Francisco'}
2,90,{'city': 'Singapore'}
3,90,{'city': 'Denver'}
4,90,{'city': 'San Francisco'}
5,90,{'city': 'Palo Alto'}
6,90,{'city': 'Chicago'}
7,90,{'city': 'San Francisco'}
8,90,{'city': 'Irving'}
9,90,{'city': 'South San Francisco'}


Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [5]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city": {'$eq': "New York"}}
project1 = {"_id":0,"name":1,"offices.city": 1,"number_of_employees": 1,"offices.latitude":1,"offices.longitude":1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)#.skip(1).limit(1)
results1

In [6]:
office_location = list(results1)
office_location

[{'name': 'Aleri',
  'number_of_employees': 90,
  'offices': {'city': 'New York',
   'latitude': 40.7592189,
   'longitude': -73.9783534}}]

In [7]:
pd.DataFrame.from_dict(office_location)

,name,number_of_employees,offices
0,Aleri,90,"{'city': 'New York', 'latitude': 40.7592189, '..."


## OFFICE LOCATION

In [8]:
inicial_lat = 40.7592189
inicial_long = -73.9783534

In [9]:
map_1 = folium.Map(location = [inicial_lat, inicial_long], zoom_start = 16)
map_1

In [10]:
office_loc = folium.Marker(location = [inicial_lat, inicial_long], tooltip = "Office location proposal")
# tooltip nos sirve para crear texto en los marcadores
office_loc.add_to(map_1)
map_1

icono = Icon(color = "blue",
             prefix = "fa",
             icon = "building-o",
             icon_color = "black",
             tooltip = "Office location proposal")

In [11]:
office_location = [inicial_lat, inicial_long]
marker_office = Marker(location = office_location, icon = icono)
marker_office.add_to(map_1)
map_1

In [12]:
load_dotenv()

True

In [13]:
city = "New York"
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

In [14]:
donde = "New York"
data = requests.get(f"https://geocode.xyz/{donde}?json=1").json()

In [15]:
print(data)

{'standard': {'addresst': {}, 'city': 'New York', 'prov': 'US', 'countryname': 'United States of America', 'postal': {}, 'confidence': '0.90'}, 'longt': '-73.95861', 'alt': {'loc': {'longt': '-73.9536869565217', 'prov': 'NY', 'city': 'NEW YORK', 'postal': '10075', 'score': '25', 'latt': '40.7724386956522'}}, 'elevation': {}, 'latt': '40.68908'}


In [16]:
nyc = geocode(city)

In [17]:
nyc

{'type': 'Point', 'coordinates': [-73.95861, 40.68908]}

In [18]:
print (data["latt"])
print (data["longt"])

40.68908
-73.95861


In [19]:
nyc = {'type': 'Point', 'coordinates': [40.68908, -73.95861]}

In [20]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

### Búsqueda de Starbucks

In [21]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [50]:
parametros = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"Starbucks",
              "limit": 100  
}

In [51]:
resp = requests.get(url= url_query, params=parametros)
data = json.loads(resp.text)

In [52]:
data.keys()

dict_keys(['meta', 'response'])

In [53]:
decoding_data = data.get("response")

In [54]:
for k,v in decoding_data.items():
    print(k,v,"\n")

suggestedFilters {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}, {'name': '$-$$$$', 'key': 'price'}]} 

suggestedRadius 3030 

headerLocation Bedford-Stuyvesant 

headerFullLocation Bedford-Stuyvesant, Brooklyn 

headerLocationGranularity neighborhood 

query starbucks 

totalResults 54 

suggestedBounds {'ne': {'lat': 40.73364744674186, 'lng': -73.93773164706342}, 'sw': {'lat': 40.65215688348848, 'lng': -74.0191341120446}} 

groups [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5bc79a2b6f0aa20039df0cde', 'name': 'Starbucks', 'contact': {}, 'location': {'address': '325 Lafayette St', 'lat': 40.6888175434886, 'lng': -73.96267818028298, 'labeledLatLngs': [{'label': 'display', 'lat': 40.6888175434886, 'lng': -73.96267818028298}], 'distance': 344, 'postalCode': '11238', 'cc': 'US', 'city': 'Ne

In [55]:
decoded = decoding_data.get("groups")[0]

In [56]:
starbucks = decoded.get("items")

In [57]:
for k,v in starbucks[0].items():
    print(k,"-->",v,"\n")

reasons --> {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]} 

venue --> {'id': '5bc79a2b6f0aa20039df0cde', 'name': 'Starbucks', 'contact': {}, 'location': {'address': '325 Lafayette St', 'lat': 40.6888175434886, 'lng': -73.96267818028298, 'labeledLatLngs': [{'label': 'display', 'lat': 40.6888175434886, 'lng': -73.96267818028298}], 'distance': 344, 'postalCode': '11238', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['325 Lafayette St', 'New York, NY 11238', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tipCount': 0, 'usersCount': 0, 'checkinsCount': 0, 'visitsCount': 0}, 'beenHere': {'count': 0, 'lastCheckinExpiredAt': 0

In [58]:
map_starbucks = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

In [59]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [60]:
print(getFromDict(starbucks[0],map_starbucks))

Starbucks


In [61]:
starbucks_nyc = []
for dic in starbucks:
    paralista = {}
    paralista["name"] = getFromDict(dic,map_starbucks)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    starbucks_nyc.append(paralista)

In [62]:
starbucks_nyc[0]

{'name': 'Starbucks',
 'latitud': 40.6888175434886,
 'longitud': -73.96267818028298}

In [63]:
dataframefinal = pd.DataFrame(starbucks_nyc)
dataframefinal.head()

,name,latitud,longitud
0,Starbucks,40.688818,-73.962678
1,Starbucks,40.683825,-73.976577
2,Starbucks,40.682636,-73.975945
3,Starbucks,40.671128,-73.957784
4,Starbucks,40.700060,-73.941432


In [64]:
gdf = gpd.GeoDataFrame(dataframefinal, geometry = gpd.points_from_xy(dataframefinal.longitud,dataframefinal.latitud))
gdf.head()

,name,latitud,longitud,geometry
0,Starbucks,40.688818,-73.962678,POINT (-73.96268 40.68882)
1,Starbucks,40.683825,-73.976577,POINT (-73.97658 40.68382)
2,Starbucks,40.682636,-73.975945,POINT (-73.97594 40.68264)
3,Starbucks,40.671128,-73.957784,POINT (-73.95778 40.67113)
4,Starbucks,40.700060,-73.941432,POINT (-73.94143 40.70006)


In [65]:
#!pip3 install cartoframes

In [66]:
Map(Layer(gdf, popup_hover = [popup_element("name","Starbucks in NYC")]))

### Búsqueda de Veganos

In [81]:
url2 = 'https://api.foursquare.com/v2/venues/explore'
vegan = "4bf58dd8d48988d1d3941735"

In [133]:
parametros_vegan = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"vegans",
              #"limit": 2000 
}

In [134]:
resp2 = requests.get(url = url_query, params = parametros_vegan)
data2 = json.loads(resp2.text)

In [135]:
data2.keys()

dict_keys(['meta', 'response'])

In [136]:
decoding_data2 = data2.get("response")

In [137]:
for k,v in decoding_data2.items():
    print(k,v,"\n")

suggestedFilters {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}, {'name': '$-$$$$', 'key': 'price'}]} 

suggestedRadius 1010 

headerLocation Bedford-Stuyvesant 

headerFullLocation Bedford-Stuyvesant, Brooklyn 

headerLocationGranularity neighborhood 

query vegans 

totalResults 89 

suggestedBounds {'ne': {'lat': 40.712521202180774, 'lng': -73.93819901789988}, 'sw': {'lat': 40.670110672727596, 'lng': -73.98354309438571}} 

groups [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4f528efa1081d6d0080f4a01', 'name': 'Clementine Bakery', 'contact': {}, 'location': {'address': '299 Greene Ave', 'crossStreet': 'btwn Classon Ave & Franklin Ave', 'lat': 40.68747316743352, 'lng': -73.95974755126716, 'labeledLatLngs': [{'label': 'display', 'lat': 40.68747316743352, 'lng': -73.95974755126716}], 'dist

In [138]:
decoded2 = decoding_data2.get("groups")[0]

In [139]:
vegan = decoded2.get("items")

In [140]:
for k,v in vegan[0].items():
    print(k,"-->",v,"\n")

reasons --> {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]} 

venue --> {'id': '4f528efa1081d6d0080f4a01', 'name': 'Clementine Bakery', 'contact': {}, 'location': {'address': '299 Greene Ave', 'crossStreet': 'btwn Classon Ave & Franklin Ave', 'lat': 40.68747316743352, 'lng': -73.95974755126716, 'labeledLatLngs': [{'label': 'display', 'lat': 40.68747316743352, 'lng': -73.95974755126716}], 'distance': 203, 'postalCode': '11238', 'cc': 'US', 'city': 'Brooklyn', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['299 Greene Ave (btwn Classon Ave & Franklin Ave)', 'Brooklyn, NY 11238', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d16a941735', 'name': 'Bakery', 'pluralName': 'Bakeries', 'shortName': 'Bakery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tipCount': 0, 'usersCount': 0, 'checkinsCount': 

In [141]:
map_vegan = ["venue","name"]
m_latitudvegan = ["venue","location","lat"]
m_longitudvegan = ["venue","location","lng"]

In [142]:
def getFromDict(diccionario2,mapa2):
    return reduce (operator.getitem,mapa2,diccionario2)

In [143]:
print(getFromDict(vegan[0],map_vegan))

Clementine Bakery


In [144]:
vegan_nyc = []
for dic in vegan:
    paralista2 = {}
    paralista2["name"] = getFromDict(dic,map_vegan)
    paralista2["latitud"] = getFromDict(dic,m_latitudvegan)
    paralista2["longitud"] = getFromDict(dic,m_longitudvegan)
    vegan_nyc.append(paralista2)

In [145]:
vegan_nyc[0]

{'name': 'Clementine Bakery',
 'latitud': 40.68747316743352,
 'longitud': -73.95974755126716}

In [146]:
dataframefina2 = pd.DataFrame(vegan_nyc)
dataframefina2.head()

,name,latitud,longitud
0,Clementine Bakery,40.687473,-73.959748
1,Cinnamon Girl,40.689343,-73.951367
2,Stonefruit Espresso + Kitchen,40.688832,-73.955061
3,Olea,40.687716,-73.970594
4,Samurai Papa,40.689791,-73.951020


In [147]:
gdf2 = gpd.GeoDataFrame(dataframefina2, geometry = gpd.points_from_xy(dataframefina2.longitud,dataframefina2.latitud))
gdf2.head()

,name,latitud,longitud,geometry
0,Clementine Bakery,40.687473,-73.959748,POINT (-73.95975 40.68747)
1,Cinnamon Girl,40.689343,-73.951367,POINT (-73.95137 40.68934)
2,Stonefruit Espresso + Kitchen,40.688832,-73.955061,POINT (-73.95506 40.68883)
3,Olea,40.687716,-73.970594,POINT (-73.97059 40.68772)
4,Samurai Papa,40.689791,-73.951020,POINT (-73.95102 40.68979)


In [148]:
Map(Layer(gdf2, popup_hover = [popup_element("name","Vegan restaurants in NYC")]))